In [7]:
import math
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [12]:
np.set_printoptions(precision=3, suppress=True)

In [13]:
marg_90 = pd.read_csv("Marginacion_1990.csv")

In [14]:
variables = marg_90[['ANALF','SPRIM','OVSDE','OVSEE','OVSAE','VHAC','OVPT','PL.5000','PO2SM']]
#variables_np = variables.to_numpy()
var_cov_std = StandardScaler().fit_transform(variables)
var_cov = np.cov(var_cov_std.T)
var_cov.shape

(9, 9)

In [16]:
def sign(x):
    """
    Helper function for computing sign of real number x.
    """
    if x >=0:
        return 1
    else:
        return -1

In [15]:
def compute_cos_sin_Jacobi_rotation(Ak, idx1, idx2):
    """
    Helper function for computing entries of Jacobi rotation.
    Args:
        Ak (numpy ndarray): Matrix of iteration k in Jacobi rotation method.
        idx1 (int): index for rows in Jacobi rotation matrix.
        idx2 (int): index for columns in Jacobi rotation matrix.
    Returns:
        c (float): value of cos of theta for Jacobi rotation matrix.
        s (float): value of sin of theta for Jacobi rotation matrix.
    """
    if np.abs(Ak[idx1,idx2]) >= np.finfo(float).eps:
        tau = (Ak[idx2, idx2] - Ak[idx1, idx1])/(2*Ak[idx1, idx2])
        t_star = sign(tau)/(np.abs(tau) + np.sqrt(1+tau**2))
        c = 1/np.sqrt(1+t_star**2)
        s = c*t_star
    else: #no rotation is performed
        c = 1
        s = 0
    return (c,s)

In [17]:
def off(A):
    """
    Frobenius norm without the main diagonal
    Args:
        Ak (numpy ndarray): Matrix for getting the Frobenius norm.
    Returns:
        s: The Frobenius norm without the sum of the main diagonal indices.
    """
    s = 0
    for i in range(A.shape[0]):
        for j in range(A.shape[1]):
            if j != i:
                s1 = math.sqrt(A[i,j]**2)
                s = s + s1
    return s

In [18]:
def compute_Jacobi_rotation(Ak, idx1, idx2):
    """
    Compute Jacobi rotation matrix.
    Args:
        Ak (numpy ndarray): Matrix of iteration k in Jacobi rotation method.
        idx1 (int): index for rows in Jacobi rotation matrix.
        idx2 (int): index for columns in Jacobi rotation matrix.
    Returns:
        J (numpy ndarray): Jacobi rotation matrix.
    """
    c,s = compute_cos_sin_Jacobi_rotation(Ak, idx1, idx2)
    m,n = Ak.shape
    J = np.eye(m)
    J[idx1, idx1] = J[idx2, idx2] = c
    J[idx1, idx2] = s
    J[idx2, idx1] = -s
    return J

In [23]:
def compute_Jacobi_egvalues(A, sweeps):
    """
    Compute Jacobi rotation matrix.
    Args:
        Ak (numpy ndarray): Matrix of iteration k in Jacobi rotation method.
        idx1 (int): index for rows in Jacobi rotation matrix.
        idx2 (int): index for columns in Jacobi rotation matrix.
    Returns:
        J (numpy ndarray): Jacobi rotation matrix.
    """
    Ak = A.copy()
    for n in range(sweeps):
        for i in range(Ak.shape[0]):
            j = i + 1
            for m in range(Ak.shape[0]-i-1):
                if off(Ak) > (1e-8)*np.linalg.norm(Ak):
                    J = compute_Jacobi_rotation(Ak, i, j)
                    Ak = J.T@Ak@J
                    j += 1
                    
    print(Ak)

In [35]:
def compute_Jacobi_egvectors(A, sweeps):
    """
    Compute Jacobi rotation matrix.
    Args:
        Ak (numpy ndarray): Matrix of iteration k in Jacobi rotation method.
        idx1 (int): index for rows in Jacobi rotation matrix.
        idx2 (int): index for columns in Jacobi rotation matrix.
    Returns:
        J (numpy ndarray): Jacobi rotation matrix.
    """
    m,n = A.shape
    Q = np.eye(m)
    for n in range(sweeps):
        for i in range(A.shape[0]):
            j = i + 1
            for m in range(A.shape[0]-i-1):
                if off(A) > (1e-8)*np.linalg.norm(A):
                    J = compute_Jacobi_rotation(A, i, j)
                    Qk = Q@J
                    Q = Qk
                    j += 1
                    
    return Qk

In [21]:
values, vectors = np.linalg.eig(var_cov)

In [38]:
compute_Jacobi_egvectors(var_cov,6)

array([[ 0.133, -0.333,  0.298,  0.387, -0.39 , -0.093,  0.163,  0.635,
        -0.205],
       [ 0.095,  0.813,  0.357, -0.064,  0.009, -0.099, -0.262,  0.264,
        -0.224],
       [ 0.079, -0.279,  0.479, -0.715,  0.128, -0.175,  0.004,  0.197,
         0.299],
       [-0.21 , -0.167, -0.068,  0.321,  0.665, -0.271, -0.434,  0.315,
         0.133],
       [ 0.266, -0.209,  0.341,  0.031,  0.438,  0.551, -0.016, -0.152,
        -0.5  ],
       [ 0.13 ,  0.143, -0.151,  0.026, -0.004,  0.692, -0.058,  0.394,
         0.549],
       [-0.075,  0.238,  0.209,  0.227,  0.386, -0.101,  0.797, -0.006,
         0.221],
       [ 0.684,  0.035, -0.527, -0.202,  0.208, -0.229,  0.17 ,  0.24 ,
        -0.171],
       [ 0.601, -0.024,  0.299,  0.371, -0.081, -0.181, -0.221, -0.39 ,
         0.413]])

In [34]:
vectors

array([[-0.356,  0.071, -0.241, -0.266, -0.674,  0.26 , -0.343, -0.299,
        -0.106],
       [-0.355, -0.006, -0.124,  0.265,  0.342, -0.127, -0.145, -0.153,
        -0.782],
       [-0.336, -0.368, -0.102,  0.365,  0.332,  0.311, -0.222, -0.327,
         0.495],
       [-0.317,  0.509,  0.429,  0.099,  0.1  ,  0.237, -0.395,  0.466,
         0.093],
       [-0.329,  0.293,  0.391, -0.19 ,  0.067, -0.515,  0.15 , -0.53 ,
         0.21 ],
       [-0.33 , -0.151, -0.457, -0.377,  0.179, -0.47 , -0.212,  0.421,
         0.207],
       [-0.34 ,  0.3  , -0.287, -0.221,  0.208,  0.429,  0.659,  0.046,
         0.012],
       [-0.343, -0.077, -0.022,  0.608, -0.477, -0.268,  0.357,  0.268,
         0.082],
       [-0.289, -0.63 ,  0.537, -0.344, -0.046,  0.15 ,  0.163,  0.177,
        -0.175]])